## Tasking Manager Monthly Report

In [1]:
import base64
import requests, json
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import interact
import ipywidgets as widgets

from itsdangerous import URLSafeTimedSerializer
from IPython.display import display
from csv import writer


In [2]:
TM_APP_BASE_URL = 'https://tasking-manager-tm4-production-api.hotosm.org'
#change for user token 
TOKEN_USER = 'Token T0RNME9EZ3ouWHpvSmRnLlN3SmV0Y012NmI1NFcxcWVPbXNiUjFDZkFVTQ=='

In [3]:
headers = {
        'Authorization': TOKEN_USER,
        'Accept-Language': 'en'
        }
columns = ['project_id','project_name','project_organisation','teams','percentege_mapped',
          'percentege_validated','id_presets','instructions']
mapping_level = { "BEGINNER": 1, "INTERMEDIATE": 2, "ADVANCED": 3}
team_roles = {"MAPPER": 0, "VALIDATOR": 1, "PROJECT_MANAGER": 2}
table = []
statistics_header = ["projectId",
  "projectArea(in sq.km)",
  "totalMappers",
  "totalTasks",
  "totalComments",
  "totalMappingTime",
  "totalValidationTime",
  "totalTimeSpent",
  "averageMappingTime",
  "averageValidationTime",
  "percentMapped",
  "percentValidated",
  "percentBadImagery",
  "aoiCentroid",
  "timeToFinishMapping",
  "timeToFinishValidating",
]

In [4]:
def append_list_as_row(file_name, list_of_elem):
    with open(file_name, 'a+', newline='') as write_obj:
        csv_writer = writer(write_obj)
        csv_writer.writerow(list_of_elem)

In [5]:
def get_project_id(project_id):
    data_result={
        'project_id':[],
        'project_name':[],
        'project_organisation':[],
        'percentege_mapped':[],
        'percentege_validated':[],
        'id_presets':[],
        'instructions':[],
        'teams':[]
    }  
    result = requests.get(TM_APP_BASE_URL + '/api/v2/projects/{project_id}/'.format(project_id=project_id), headers=headers).json()
    data_result['project_id'].append(project_id)
    data_result['project_name'].append(result['projectInfo']['name'])
    data_result['project_organisation'].append(result['organisationName'])
    data_result['percentege_mapped'].append(result['percentMapped'])
    data_result['percentege_validated'].append(result['percentValidated'])
    data_result['id_presets'].append(result['idPresets'])
    data_result['instructions'].append(len(result['projectInfo']['instructions']))
    teams = [team["name"] for team in result["teams"]]
    data_result['teams'].append(teams)    
    return data_result

In [6]:
def btn_eventhandler(object):    
    result = get_project_id(project_id.value)
    table.append(result)

In [7]:
project_id = widgets.Text(value="8469")  #default value 9143
display(project_id)
btn = widgets.Button(description='Results')
btn.on_click(btn_eventhandler)
display(btn)

Text(value='8469')

Button(description='Results', style=ButtonStyle())

In [9]:
df = pd.DataFrame(table[0])
df.head()

,project_id,project_name,project_organisation,percentege_mapped,percentege_validated,id_presets,instructions,teams
0,8469,"COVID-19 - Gao - Gao, Mali",HOT,100,100,[building],2850,"[Validator Trainees, HOT Global Validators]"


## For mappers:
####  The current number of projects is 

In [ ]:
def get_contries():
    countries = requests.get(TM_APP_BASE_URL + '/api/v2/countries/', headers=headers).json()
    return countries['tags']

def get_all_projects(country):
    params = {'country': country}
    projects = requests.get(TM_APP_BASE_URL + '/api/v2/projects/', headers=headers, params=params).json()
    return projects

def total_projects_by_country():
    countries = get_contries()
    row = []
    for country in countries:
        total = 0
        projects = get_all_projects(country)
        if projects['results']:
            total = projects['pagination']['total']
        row = [country, total]
        append_list_as_row('projects-by-country.csv', row)

total_projects_by_country()

In [ ]:
# number of projects by mapper level
data_mappers = []
def get_total_of_projects(mapper_level):
    params = {"mapperLevel": mapper_level}
    result = requests.get(
        TM_APP_BASE_URL + "/api/v2/projects", headers=headers, params=params
    ).json()
    return result["pagination"]["total"]
    
    
def get_number_of_projects_by_mapper_level():
    mapper_level = ["BEGINNER","INTERMEDIATE","ADVANCED"]
    mappers = {}
    for level in mapper_level:
        mappers.update({level: get_total_of_projects(level)})
    return mappers        

In [ ]:
users = get_number_of_projects_by_mapper_level()
df = pd.DataFrame([users])
df.head()

## For validators:

In [ ]:
data_teams = []
members = []

def get_validators(team_role):
    params = {"team_role": team_role}
    result = requests.get(
        TM_APP_BASE_URL + "/api/v2/teams/", headers=headers, params=params
    ).json()
    for team in result["teams"]:
        team_detail = requests.get(
            TM_APP_BASE_URL + "/api/v2/teams/{team_id}/".format(team_id=team["teamId"]),
            headers=headers,
        ).json()
        projects = team_detail["team_projects"]
        members.append({"teamId": team["teamId"], "members": team["members"]})
        
        for project in projects:            
            try:
                statistics = requests.get(
                    TM_APP_BASE_URL
                    + "/api/v2/projects/{project_id}/statistics/".format(
                        project_id=project["project_id"]
                    ),
                    headers=headers,
                ).json()
                project_detail = {
                        "teamId":team["teamId"],
                        "team_name": team["name"],
                        "project_id": int(project["project_id"]),
                        "project_name": project["project_name"],
                        "tasks": int(statistics["totalTasks"]),
                        "percentMapped": int(statistics["percentMapped"]),
                        "percentValidated": int(statistics["percentValidated"]),
                    }                
                append_list_as_row(
                    "validator.csv", project_detail.values()
                )
            except:
                pass
        
    print("------Export completed------")


get_validators("VALIDATORS")


In [ ]:
#number of mappers grouped by experience
user_experience = {}
def get_user_detail(username):
    user =  requests.get(TM_APP_BASE_URL + '/api/v2/users/queries/{username}/'.format(username=username),
                         headers=headers).json()                
    return {'id': user['id'],
                 'username': user['username'],
                 'role': user['role'],
                 'mappingLevel': user['mappingLevel'],
                 'country': user["country"]
        }

def user_by_experience():
    beginner = 0 
    intermediate = 0
    advanced = 0 

    for row in members:
        for member in row['members']:
            user = get_user_detail(member['username'])
            if user['mappingLevel'] == 'BEGINNER':
                beginner += 1
            elif user['mappingLevel'] == 'INTERMEDIATE':
                intermediate += 1 
            else: 
                advanced += 1
    return {
        'BEGINNER': beginner,
        'INTERMEDIATE': intermediate,
        'ADVANCED': advanced
    }
            

In [ ]:
users = user_by_experience()
df = pd.DataFrame([users])
df.head()

## for project managers
### Statistics on project


In [ ]:
def get_project_stast(project_id):
    statistics = requests.get(TM_APP_BASE_URL + 
                              '/api/v2/projects/{project_id}/statistics/'.format(project_id=project_id),
                              headers=headers).json()          
    return statistics

def get_all_projects():
    append_list_as_row('statistics_project.csv', statistics_header)
    projects = requests.get(TM_APP_BASE_URL + '/api/v2/projects/', headers=headers).json()
    for page in range(1, projects['pagination']['pages']):
        params = {'page': page}
        project_list = requests.get(TM_APP_BASE_URL + '/api/v2/projects/', headers=headers, params=params).json()
        for project in project_list['results']:
            project_id = project['projectId']
            try:
                statistics = get_project_stast(project_id)
                append_list_as_row('statistics_project.csv', statistics.values())
            except:
                pass
get_all_projects()